# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Jun  2 2022 10:53AM  238645        19  ADV80002997  AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:53AM  238645        19  ADV80003000  AdvaGen Pharma, Ltd   
2  Jun  2 2022 10:53AM  238645        19  ADV80003001  AdvaGen Pharma, Ltd   
3  Jun  2 2022 10:53AM  238645        19  ADV80003002  AdvaGen Pharma, Ltd   
4  Jun  2 2022 10:53AM  238645        19  ADV80003017  AdvaGen Pharma, Ltd   
5  Jun  2 2022 10:53AM  238645        19  ADV80003018  AdvaGen Pharma, Ltd   
6  Jun  2 2022 10:53AM  238645        19  ADV80003019  AdvaGen Pharma, Ltd   
7  Jun  2 2022 10:53AM  238645        19  ADV80003020  AdvaGen Pharma, Ltd   
8  Jun  2 2022 10:53AM  238645        19  ADV80003021  AdvaGen Pharma, Ltd   
9  Jun  2 2022 10:53AM  238645        19  ADV80003022  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
51  238639       Released          1
52  238641       Released         18
53  238643       Released         17
54  238644       Released          1
55  238645       Released         10

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238639                NaN        NaN       1.0
238641                NaN        NaN      18.0
238643                NaN        NaN      17.0
238644                NaN        NaN       1.0
238645                NaN        NaN      10.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238529                0.0        2.0      58.0
238544                0.0       17.0       4.0
238551                0.0        0.0       1.0
238553                0.0        1.0       0.0
238556                0.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238529                  0          2        58
238544                  0         17         4
238551                  0          0         1
238553                  0          1         0
238556                  0          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238529          0          2        58
1               238544          0         17         4
2               238551          0          0         1
3               238553          0          1         0
4               238556          0          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               238529                   2       58
1               238544                  17        4
2               238551                            1
3               238553                   1         
4               238556                            2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0    Jun  2 2022 10:53AM  238645        19              AdvaGen Pharma, Ltd
10   Jun  2 2022 10:49AM  238644        16   Sartorius Bioprocess Solutions
11   Jun  2 2022 10:25AM  238643        10                ISDIN Corporation
28   Jun  2 2022 10:20AM  238641        10                ISDIN Corporation
46   Jun  2 2022 10:10AM  238633        15             Person & Covey, Inc.
63   Jun  2 2022 10:07AM  238639        16  American International Chemical
64   Jun  2 2022 10:05AM  238631        15             Person & Covey, Inc.
78   Jun  2 2022 10:04AM  238564        20         ACI Healthcare USA, Inc.
85   Jun  2 2022 10:03AM  238628        10                ISDIN Corporation
126  Jun  2 2022 10:03AM  238626        10                ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Jun  2 2022 10:53AM  238645        19              AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:49AM  238644        16   Sartorius Bioprocess Solutions   
2  Jun  2 2022 10:25AM  238643        10                ISDIN Corporation   
3  Jun  2 2022 10:20AM  238641        10                ISDIN Corporation   
4  Jun  2 2022 10:10AM  238633        15             Person & Covey, Inc.   
5  Jun  2 2022 10:07AM  238639        16  American International Chemical   
6  Jun  2 2022 10:05AM  238631        15             Person & Covey, Inc.   
7  Jun  2 2022 10:04AM  238564        20         ACI Healthcare USA, Inc.   
8  Jun  2 2022 10:03AM  238628        10                ISDIN Corporation   
9  Jun  2 2022 10:03AM  238626        10                ISDIN Corporation   

  Completed Executing Released  
0                           10  
1                            1  
2                           17  
3                           18  
4                  10        7  
5                            1  
6                   5        9  
7                            7  
8                           41  
9                           41

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Jun  2 2022 10:53AM  238645        19              AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:49AM  238644        16   Sartorius Bioprocess Solutions   
2  Jun  2 2022 10:25AM  238643        10                ISDIN Corporation   
3  Jun  2 2022 10:20AM  238641        10                ISDIN Corporation   
4  Jun  2 2022 10:10AM  238633        15             Person & Covey, Inc.   
5  Jun  2 2022 10:07AM  238639        16  American International Chemical   
6  Jun  2 2022 10:05AM  238631        15             Person & Covey, Inc.   
7  Jun  2 2022 10:04AM  238564        20         ACI Healthcare USA, Inc.   
8  Jun  2 2022 10:03AM  238628        10                ISDIN Corporation   
9  Jun  2 2022 10:03AM  238626        10                ISDIN Corporation   

  Released Executing Completed  
0       10                      
1        1                      
2       17                      
3       18                      
4        7        10            
5        1                      
6        9         5            
7        7                      
8       41                      
9       41

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun  2 2022 10:53AM  238645        19             AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:49AM  238644        16  Sartorius Bioprocess Solutions   
2  Jun  2 2022 10:25AM  238643        10               ISDIN Corporation   
3  Jun  2 2022 10:20AM  238641        10               ISDIN Corporation   
4  Jun  2 2022 10:10AM  238633        15            Person & Covey, Inc.   

  Released Executing Completed  
0       10                      
1        1                      
2       17                      
3       18                      
4        7        10

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun  2 2022 10:53AM  238645        19             AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:49AM  238644        16  Sartorius Bioprocess Solutions   
2  Jun  2 2022 10:25AM  238643        10               ISDIN Corporation   
3  Jun  2 2022 10:20AM  238641        10               ISDIN Corporation   
4  Jun  2 2022 10:10AM  238633        15            Person & Covey, Inc.   

   Released  Executing  Completed  
0      10.0        NaN        NaN  
1       1.0        NaN        NaN  
2      17.0        NaN        NaN  
3      18.0        NaN        NaN  
4       7.0       10.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Jun  2 2022 10:53AM  238645        19             AdvaGen Pharma, Ltd   
1  Jun  2 2022 10:49AM  238644        16  Sartorius Bioprocess Solutions   
2  Jun  2 2022 10:25AM  238643        10               ISDIN Corporation   
3  Jun  2 2022 10:20AM  238641        10               ISDIN Corporation   
4  Jun  2 2022 10:10AM  238633        15            Person & Covey, Inc.   

   Released  Executing  Completed  
0      10.0        0.0        0.0  
1       1.0        0.0        0.0  
2      17.0        0.0        0.0  
3      18.0        0.0        0.0  
4       7.0       10.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2624791     145.0       17.0        1.0
12         1431504      67.0        5.0        0.0
15         1908929      70.0       22.0        0.0
16          715836       2.0        1.0        0.0
19         1192989      31.0       38.0       13.0
20         2385848      18.0       18.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2624791     145.0       17.0        1.0
1        12  1431504      67.0        5.0        0.0
2        15  1908929      70.0       22.0        0.0
3        16   715836       2.0        1.0        0.0
4        19  1192989      31.0       38.0       13.0
5        20  2385848      18.0       18.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     145.0       17.0        1.0
1        12      67.0        5.0        0.0
2        15      70.0       22.0        0.0
3        16       2.0        1.0        0.0
4        19      31.0       38.0       13.0
5        20      18.0       18.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  145.0
1        12  Released   67.0
2        15  Released   70.0
3        16  Released    2.0
4        19  Released   31.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   16    19    20
Status                                       
Completed    1.0   0.0   0.0  0.0  13.0   0.0
Executing   17.0   5.0  22.0  1.0  38.0  18.0
Released   145.0  67.0  70.0  2.0  31.0  18.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   16    19    20
0          Completed    1.0   0.0   0.0  0.0  13.0   0.0
1          Executing   17.0   5.0  22.0  1.0  38.0  18.0
2           Released  145.0  67.0  70.0  2.0  31.0  18.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   16    19    20
0  Completed    1.0   0.0   0.0  0.0  13.0   0.0
1  Executing   17.0   5.0  22.0  1.0  38.0  18.0
2   Released  145.0  67.0  70.0  2.0  31.0  18.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()